Assignment 14

In [90]:
import numpy as np
import cv2
import itertools
import collections
 
def GetTransforms(dataset):
    # Get data from dataset
    data = []
    with open(dataset, 'r') as f:
        data = f.readlines()
    # strip the new line character
    data = [x.strip() for x in data]

    beginSequence = data[0]

    transforms = {}
    for i in range(2, len(data)):
        split = data[i].split(' -> ')
        if len(split) == 2:
            base = split[0]
            insert = split[1]
            transforms[hash(base)] = insert

    
    return beginSequence, transforms 

def PerformPolimarize(base, transforms, count):
    # Perforamce the polimarization
    if count == 0:
        return base
    else:
        newBase = ''
        for i in range(1, len(base)):
            #print(transforms[j].base, transforms[j].insert)
            newBase += base[i-1] + transforms[hash(base[i-1:i+1])]
                    
        newBase += base[-1]
    
        #print(f'polemer lenght: {len(newBase)}')
        #print(f'steps to do: {count}')
        return PerformPolimarize(newBase, transforms, count-1) 


    
            


In [91]:
start, transforms = GetTransforms('./data/aoc14.txt')
print(start)
print(PerformPolimarize(start, transforms, 1))

newPolimer = PerformPolimarize(start, transforms, 10)

# find all the unique characters
unique = set(newPolimer)

# Count the number of times each character appears
counts = {}
for i in unique:
    counts[i] = newPolimer.count(i)
print(counts)

# Find the character with the lowest count
minCount = min(counts.values())

# Find the character with the highest count
maxCount = max(counts.values())

print( maxCount - minCount)



FNFPPNKPPHSOKFFHOFOC
FONSFHPNPKNVKKPNPHHFSOOCKCFBFOHPOCFCOCC
{'N': 2885, 'S': 530, 'H': 2324, 'P': 1137, 'C': 3505, 'F': 2454, 'O': 1431, 'K': 840, 'B': 2253, 'V': 2098}
2975


In [92]:
# create all return cominations of a cominatiiion of x characters
def GetCombinations(x, charcterset):
    combs = []
    print(characterset)
    for i in range(2, x+2):
        print(i)
        combs += list(itertools.product(charcterset, repeat= i))
    return combs

def PreComputedTransforms(base,transforms, count =1):

    return PerformPolimarize(base, transforms, count)[:-1]
    


def PerformAcceleratedPolimarize(base, preComputedTransforms, transforms, count, maxCharCount, printData =True):
    # Perforamce the polimarization
    if count == 0:
        return base
    else:
        newBase = ''
        # split base into chunks of maxCharCount if possible
        chunks = []
        for i in range(0, len(base), maxCharCount):
            if i + maxCharCount > len(base):
                chunks.append(base[i:])
            else:
                chunks.append( base[i:i+maxCharCount+1])
        #print(base)
        #print(chunks)
        #print(f'Chucks: {len(chunks)}')
        persentage = 0

        for i in range(0,len(chunks)):
            try: 
                newBase += preComputedTransforms[hash(chunks[i])]
            except:
                newBase += PerformPolimarize(chunks[i], transforms, 1)[:-1]
            if ((i+1) % 100 == 0 and printData):
                print(f'{(i+1)/len(chunks)}%')
        newBase += base[-1]
        if(printData):
            print(f'polemer lenght: {len(newBase)}')
            print(f'steps to do: {count}')
        return PerformAcceleratedPolimarize(newBase, preComputedTransforms, transforms, count-1, maxCharCount, printData)

def PerformDepthAcceleratedPolimarize(base, preComputedTransforms, printData =True):
    # chucks is a numpy array if intgers of lenght of base +1
    chunks = np.zeros(len(base)-1, dtype= np.int64)
    #print(base)
    for i in range(0,len(base)-1):
        #print(base[i:i+2])
        chunks[i]= hash(base[i:i+2])
        if ((i+1) % 100 == 0 and printData):
                print(f'{(i+1)/len(base)*100}%')
        
    return   chunks 




In [93]:
start, transforms = GetTransforms('./data/aoc14.txt')

#set to string 
tmpset = set(newPolimer)
characterset = ''
for i in tmpset:
    characterset += i

preComputed = {}
charLenght = 4
for i in GetCombinations(charLenght, charcterset=characterset):
    # combine to string 
    string = ''
    for j in i:
        string += j
    preComputed[hash(string)] = PreComputedTransforms(string, transforms)
    #print(string,preComputed[hash(string)])
#print(preComputed)

newPolimer =PerformAcceleratedPolimarize(start, preComputed, transforms,5, charLenght)

# find all the unique characters
unique = set(newPolimer)

# Count the number of times each character appears
counts = {}
for i in unique:
    counts[i] = newPolimer.count(i)
print(counts)

# Find the character with the lowest count
minCount = min(counts.values())

# Find the character with the highest count
maxCount = max(counts.values())

print( maxCount - minCount)

NSHPCFOKBV
2
3
4
5
polemer lenght: 39
steps to do: 5
polemer lenght: 77
steps to do: 4
polemer lenght: 153
steps to do: 3
polemer lenght: 305
steps to do: 2
polemer lenght: 609
steps to do: 1
{'S': 18, 'N': 70, 'H': 58, 'P': 45, 'C': 129, 'F': 103, 'O': 44, 'K': 44, 'B': 67, 'V': 31}
111


In [94]:
start, transforms = GetTransforms('./data/aoc14.txt')

#set to string 
tmpset = set(newPolimer)
characterset = ''
for i in tmpset:
    characterset += i


#preComputedDepth ={}
cobinations = GetCombinations(1, charcterset=characterset)
for i in range(0, len(cobinations)):

    # combine to string 
    string = ''
    for j in cobinations[i]:
        string += j
    #preComputedDepth[np.int64(hash(string))] = PerformAcceleratedPolimarize(string, preComputed, transforms, 20, charLenght, False)[:-1]
print("Start Transform")
newPolimer = start
# round 1 to 20
if(False):
    tmp =PerformDepthAcceleratedPolimarize(newPolimer, preComputedDepth, True)
    print(tmp)
    tmpPolymer = ''
    for i in range(0,len(tmp)):
        tmpPolymer += preComputedDepth[tmp[i]]
    tmpPolymer += newPolimer[-1]
    newPolimer = tmpPolymer


    # round 21 to 40
    tmp =PerformDepthAcceleratedPolimarize(newPolimer, preComputedDepth, True)




SNHPCFOKBV
2
Start Transform


In [99]:
# get all the unique integers
#uniqueCombinations = collections.Counter(tmp)
uniqueCharacters = {'S': 0, 'N': 0, 'H': 0, 'P': 0, 'C': 0, 'F': 0, 'O': 0, 'K': 0, 'B': 0, 'V': 0}
uniqueCharacters[start[-1]] += 1

key_list = list(preComputedDepth.keys())
val_list = list(preComputedDepth.values())

uniqueCharacters_keys = list(uniqueCharacters.keys())

combo = {}
collection = {}
i=0
for i in range(len(preComputedDepth)):
    collection = collections.Counter(val_list[i])
    for j in uniqueCharacters_keys:
         uniqueCharacters[j] += collection[j] * uniqueCombinations[key_list[i]] 

print(combo)


# Count the number of times each integer appears


print(unique)

{}
{'S', 'N', 'H', 'P', 'C', 'F', 'O', 'K', 'B', 'V'}


In [100]:
minCount = min(uniqueCharacters.values())

# Find the character with the highest count
maxCount = max(uniqueCharacters.values())

print( maxCount - minCount)

3015383850689


In [ ]:
combos = list(preComputedDepth.keys())
combos.sort()
print(combos)

[-9181710493297235555, -8647714121679725675, -8495393806868105532, -8487504442577322016, -8461195917661475766, -8028861853570068757, -7760669381403001234, -7719442797858719051, -7504008472732099181, -7456637135727214404, -7370022017645794479, -7213060820683071364, -7209054706945703578, -7006983167772025274, -6972224591416811530, -6629648645311243928, -6627035686185390017, -6607492453512294622, -6447248876938319060, -6351544035892720342, -6329667256536643912, -5901515212886038978, -5657987433743134714, -5516604111417745442, -5122986246781213946, -4725286663324112899, -4560371000421923108, -4492256626722576859, -4384871673408042328, -3604784015250324599, -3531459626560908588, -3075267825608387518, -2944952641993352179, -2790585838668869428, -2607574506359171166, -2137777081058470669, -2066788464915931270, -1939765976594439364, -1928496765913600971, -1842378536741254029, -1724812312876352524, -1494258228497584496, -1298533491985746614, -1133609583968170647, -1098385311010891958, -10913536